In [59]:
import numpy as np
import cv2
import serial
import config as cf
import threading
import time

import RPi.GPIO as GPIO
import time
GPIO.setmode(GPIO.BOARD)
    
for i in range(5):
    try:
        port = '/dev/ttyUSB'+str(i)
        cf.ser = serial.Serial(port,250000)
        break
    except:
        pass
    
font = cv2.FONT_HERSHEY_SIMPLEX
WIDTH = 400
HEIGHT = 400
cf.global_img = np.zeros((WIDTH, HEIGHT, 3), np.uint8)
cf.RUN = True
cf.pause = False
cf.message_arr = ["NoMessage"]

def ping_to_arduino():
    print("=>> Ping to arduino...")
    message = "PUSH /"
    t1 = time.time()
    try:
        cf.ser.write(message.encode())
    except:
        pass
    while cf.message_arr[0] != 'PULL' and (time.time() - t1)<5:
        time.sleep(0.001)
    print("=>> Ping result:", time.time() - t1)

# LEDs parameters
cf.LED_ROWS = 4
cf.LED_COLLUMS = 9
cf.LEDs = np.zeros((cf.LED_ROWS , cf.LED_COLLUMS))
cf.LEDs_threshold = np.ones((cf.LED_ROWS , cf.LED_COLLUMS))
cf.LEDs_min = np.zeros((cf.LED_ROWS , cf.LED_COLLUMS))
cf.LEDs_max = np.zeros((cf.LED_ROWS , cf.LED_COLLUMS))
cf.LEDs_MODE = np.zeros_like(cf.LEDs)
cf.LEDs_MODE_show = np.zeros_like(cf.LEDs)
cf.LEDs_weighted = np.zeros_like(cf.LEDs_MODE)
cf.CENTER_LEDs_MODE = np.zeros(cf.LED_ROWS)
cf.MAX_LEFT_LEDs = np.zeros(cf.LED_ROWS)
cf.MAX_RIGHT_LEDs = np.zeros(cf.LED_ROWS)
cf.LEDs_threshold = np.load("LEDs_threshold.npy")

#CTHTs
CTHTs = [31, 29, 35, 33, 37]
GPIO.setup(CTHTs, GPIO.IN)
cf.CTHTs = np.zeros(len(CTHTs))
cf.Counter = np.zeros(4)
cf.Counter_Mode = np.zeros((4, 3))
cf.Counter_e = [True, True, True, True]
def read_inputs():
    print("** read_inputs is running...")
    t_counter = time.time()
    while cf.RUN:
        try:
            # LEDS
            cf.message = cf.ser.readline().decode()[:-2] 
            cf.message_arr = cf.message.split(" ")
            if cf.message_arr[0] == 'LEDs':
                LEDs_arr = np.array(cf.message_arr[1:].copy()).astype(np.int32)
                LEDs_arr = LEDs_arr.reshape((cf.LED_ROWS, cf.LED_COLLUMS))
                cf.LEDs = LEDs_arr.copy()
                LEDs_MODE = np.zeros_like(cf.LEDs)
                LEDs_MODE[cf.LEDs >= cf.LEDs_threshold] = 1
                LEDs_MODE[cf.LEDs < cf.LEDs_threshold] = 0
                #LEDs_MODE[0] = np.flip(LEDs_MODE[0].copy())
                #LEDs_MODE[3] = np.flip(LEDs_MODE[3].copy())
                cf.LEDs_MODE_show = LEDs_MODE.copy()
                cf.LEDs_MODE_show[2] = np.flip(cf.LEDs_MODE_show[2])
                #LEDs_MODE[2] = np.flip(LEDs_MODE[2].copy())
                cf.LEDs_MODE = LEDs_MODE.copy()
                weight = np.array([4, 2, 0.75, 0.25, 0, 0.25, 0.75, 2, 4])
                cf.LEDs_weighted = np.multiply(cf.LEDs_MODE, weight)
                center_led = int((cf.LED_COLLUMS-1)/2)
                cf.CENTER_LEDs_MODE = cf.LEDs_MODE[:, center_led]
                cf.MAX_LEFT_LEDs =  np.max(cf.LEDs_weighted[:, :center_led], axis= 1)
                cf.MAX_RIGHT_LEDs =  np.max(cf.LEDs_weighted[:, center_led+1:], axis= 1)
                if time.time()-t_counter>0.2:
                    cf.Counter_Mode[:, 0:2] =  cf.Counter_Mode[:, 1:3].copy()
                    for i in range(4):
                        if np.sum(cf.LEDs_MODE[i])>0:
                            cf.Counter_Mode[i][2] = 1
                        else:
                            cf.Counter_Mode[i][2] = 0
                            
                        if np.sum(cf.Counter_Mode[i])==2 and cf.Counter_e[i] :
                            cf.Counter[i]+=1
                            cf.Counter_e[i] = False
                        if np.sum(cf.Counter_Mode[i])==0:
                            cf.Counter_e[i] = True
                    t_couter = time.time()
            for i in range(cf.CTHTs.shape[0]):
                cf.CTHTs[i] = GPIO.input(CTHTs[i])
        except Exception as e: 
            print (e)
    print('** read_inputs is stopped!')
    
# Speed parameters
# MOVE DIRECTION
cf.UP = [1, 1, 1, 1]
cf.DOWN = [-1, -1, -1, -1]
cf.LEFT = [-1, 1, -1, 1]
cf.RIGHT = [1, -1, 1, -1]
cf.TURN_LEFT = [-1, 1, 1, -1]
cf.TURN_RIGHT = [1, -1, -1, 1]
cf.PAUSE = [0, 0, 0, 0]
cf.cab = np.array([1 , 1, 1, 1])
cf.DIRECTIONs = [cf.UP, cf.LEFT, cf.DOWN, cf.RIGHT, cf.PAUSE]
cf.NAMES_OF_DIR = ["UP", "LEFT", "DOWN", "RIGHT"]
cf.last_message = ""


def move(direction, speed):
    if cf.pause:
        this_speed = 0
    else:
        this_speed = speed
    direction = np.multiply(cf.cab, direction)
    message = "MOV"
    for i in direction:
        message+= " " + str(int(i*this_speed))
    message+= " /"
    try:
        cf.ser.write(message.encode())
    except:
        pass
    

def move_controller():
    print('** move_controllers is running...')
    while cf.RUN:
        time.sleep(0.05)
        move(cf.direction, cf.speed)
    print('** move_controllers is stopped!')

    
def show_leds_status(img):
    ## LEDS
    dis = 20
    for i in range(cf.LED_COLLUMS):
        begin = int((WIDTH - ((cf.LED_COLLUMS-1)*dis))/2)
        cv2.circle(img, ( begin+ dis*i, dis), 7, (255, 255, 255), -1)
        if cf.LEDs_MODE_show[0, i]:
            cv2.circle(img, ( begin+ dis*i, dis), 5, (0, 255, 0), -1)
    for i in range(cf.LED_COLLUMS):
        begin = int((HEIGHT - ((cf.LED_COLLUMS-1)*dis))/2)
        cv2.circle(img, (dis,  begin+ dis*i), 7, (255, 255, 255), -1)
        if cf.LEDs_MODE_show[1, i]:
            cv2.circle(img, (dis,  begin+ dis*i), 5, (0, 255, 0), -1)
    for i in range(cf.LED_COLLUMS):
        begin = int((HEIGHT - ((cf.LED_COLLUMS-1)*dis))/2)
        cv2.circle(img, (WIDTH - dis,  begin+ dis*i), 7, (255, 255, 255), -1)
        if cf.LEDs_MODE_show[3, i]:
            cv2.circle(img, (WIDTH - dis,  begin+ dis*i), 5, (0, 255, 0), -1)
    
    for i in range(cf.LED_COLLUMS):
        begin = int((WIDTH - ((cf.LED_COLLUMS-1)*dis))/2)
        cv2.circle(img, ( begin+ dis*i, HEIGHT - dis), 7, (255, 255, 255), -1)
        if cf.LEDs_MODE_show[2, i]:
            cv2.circle(img, ( begin+ dis*i, HEIGHT - dis), 5, (0, 255, 0), -1)
            
    for i in range(cf.CTHTs.shape[0]):
        begin = int((WIDTH - ((cf.CTHTs.shape[0]-1)*dis))/2)
        cv2.circle(img, ( begin+ dis*i, HEIGHT - dis-50), 7, (255, 255, 255), -1)
        if not cf.CTHTs[i]:
            cv2.circle(img, ( begin+ dis*i, HEIGHT - dis-50), 5, (0, 255, 0), -1)
            
    for i in range(3):
        begin = int((WIDTH - (2*dis))/2)
        cv2.circle(img, ( begin+ dis*i, dis+50), 7, (255, 255, 255), -1)
        if cf.Counter_Mode[0][i]==1:
            cv2.circle(img, ( begin+ dis*i, dis+50), 5, (0, 255, 0), -1)
    
    
    for i in range(4):
        cv2.putText(img, str(int(cf.Counter[i])), (100 + i*50, dis+100), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(img, str(round(cf.direction[i],2)), (100+ i*50, 200), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(img, str(round(cf.ratio,2)), (120, 240), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
    
    
def go_with_line(values):
    direction = values[0]
    cf.pause = False
    print('=>> go_with_line:', cf.NAMES_OF_DIR[direction])
    DIRECTION = cf.DIRECTIONs[direction]
    rotate = False
    while not cf.pause and cf.RUN:
        time.sleep(0.005)
        total_weight = np.sum(cf.LEDs_MODE[direction])
        check_led = cf.CENTER_LEDs_MODE[direction]
        left_max = cf.MAX_LEFT_LEDs[direction]
        right_max = cf.MAX_RIGHT_LEDs[direction]
        minus = left_max - right_max
        if minus == 0 and check_led and total_weight!=0 and total_weight!=cf.LED_COLLUMS:
            cf.rotate = False
        if minus != 0:
            cf.rotate = True
            cf.ratio = minus
        if not cf.rotate:
            cf.direction = DIRECTION
            cf.speed = cf.default_speed
        else:
            UP_ratio = 5
            vector = (UP_ratio*np.array(DIRECTION)+cf.ratio*np.array(cf.TURN_LEFT))/(UP_ratio+abs(cf.ratio))
            cf.direction = vector
            cf.speed = cf.default_speed
    cf.speed = cf.default_speed
cf.command = ["nocommand", -1]

def set_command(command, value):
    cf.command = (command, value)
    cf.direction = [0, 0, 0, 0]
    cf.pause = True

def DecodeCommand():
    print("** DecodeCommand threading started!")
    while cf.RUN:
        time.sleep(0.1)      
        if cf.command[0] == 'go_with_line':
            go_with_line(cf.command[1])
            
        if cf.command[0] == 'run_router':
            run_router()
    print("** DecodeCommand threading stopped!")


def pause():
    cf.command = ["nocommand", -1]
    cf.pause = True
    cf.direction = [0, 0, 0, 0]
    time.sleep(0.1)
    print("pausing")

def quit():
    cf.command = ["nocommand", -1]
    cf.pause = True
    cf.RUN = False
    cf.direction = [0, 0, 0, 0]
    time.sleep(0.1)
    print("quit!")

In [ ]:
cf.RUN = True
cf.pause = False

cf.last_message = ""
cf.speed = 255
cf.default_speed = cf.speed
cf.direction = [0, 0, 0, 0]
cf.ratio = 0
cf.rotate = False

read_inputs_threading = threading.Thread(target=read_inputs)
read_inputs_threading.start()
time.sleep(0.1)
move_controller_threading = threading.Thread(target=move_controller)
move_controller_threading.start()
time.sleep(0.1)
DecodeCommand_threading = threading.Thread(target=DecodeCommand)
DecodeCommand_threading.start()
time.sleep(0.1)

while cf.RUN:
    img = np.zeros((HEIGHT, WIDTH, 3), np.uint8)
    show_leds_status(img)
    cv2.imshow("robocon", img)
    k = cv2.waitKey(1)
    if k == ord('q'):
        quit()
        break
    if k == 32:
        pause()
    if k== 56:
        cf.direction = cf.UP
        cf.pause = False
    if k== 50:
        cf.direction = cf.DOWN
        cf.pause = False
    if k== 52:
        cf.direction = cf.LEFT
        cf.pause = False
    if k== 54:
        cf.direction = cf.RIGHT
        cf.pause = False
    if k== ord('o'):
        cf.direction = cf.TURN_LEFT
        cf.pause = False
    if k== ord('p'):
        cf.direction = cf.TURN_RIGHT
        cf.pause = False
    if k== ord('r'):
        cf.pause = False
    if k == ord('z'):
        cf.LEDs_min = cf.LEDs.copy()
        print("min:") 
        print(cf.LEDs_min)
    if k == ord('x'):
        cf.LEDs_max[0] = cf.LEDs[0].copy()
        print("max UP:")
        print( cf.LEDs_max[0])
    if k == ord('j'):
        print(cf.LEDs)
        print(cf.LEDs_threshold)
        print(cf.LEDs_MODE)
    if k == ord('c'):
        cf.LEDs_max[1] = cf.LEDs[1].copy()
        print("max LEFT:")
        print( cf.LEDs_max[1])
    if k == ord('v'):
        cf.LEDs_max[2] = cf.LEDs[2].copy()
        print("max RIGHT:")
        print( cf.LEDs_max[2])
    if k == ord('b'):
        cf.LEDs_max[3] = cf.LEDs[3].copy()
        print("max BOTTOM:")
        print( cf.LEDs_max[3])
    if k== ord('w'):
        set_command("go_with_line",[0])
    if k== ord('a'):
        set_command("go_with_line", [1])
    if k== ord('d'):
        set_command("go_with_line", [3])
    if k== ord('s'):
        set_command("go_with_line", [2])
    if k== ord('i'):
        ping_to_arduino()   
    if k == ord ('n'):
        cf.LEDs_threshold = (cf.LEDs_min +cf.LEDs_max)/2
        np.save("LEDs_threshold.npy", cf.LEDs_threshold)
        print("Hieu:")
        print( -cf.LEDs_min +cf.LEDs_max)
        print("Threshold:")
        print( cf.LEDs_threshold)
cv2.destroyAllWindows()

** read_inputs is running...
** move_controllers is running...
** DecodeCommand threading started!
=>> go_with_line: DOWN
pausing
pausing
pausing
pausing
pausing
pausing
pausing
=>> go_with_line: UP
pausing
pausing
pausing
pausing
pausing
pausing
pausing
=>> go_with_line: UP
pausing
=>> go_with_line: UP
